In [1]:
using JuliaRunClient
ctx = Context()

JuliaRunClient for 17523648210908459309 @ http://juliarunremote-svc.juliarun:80

In [2]:
job = JuliaBatch("mywebserver")

JuliaRunClient.JuliaBatch("mywebserver")

In [4]:
# code for a simple webserver
webserver_script = """
using HttpServer

http = HttpHandler() do req::Request, res::Response
    Response("JuliaRun says hello from " * gethostname())
end

server = Server(http)
run(server, 8000)        
"""

open("/mnt/juliabox/helloweb.jl", "w") do f
    println(f, webserver_script)
end

In [5]:
# start a webserver job, specifying port 8000 to be opened up
@result submitJob(ctx, job; start_script="/mnt/juliabox/helloweb.jl", run_volume="juliabox", image="juliabox", cpu="1", memory="1Gi", shell="/juliabox/scripts/master.sh", ports="8000:8000")

In [6]:
# check that the job has started
@result getJobStatus(ctx, job)

Dict{String,Any} with 6 entries:
  "nworkers_succeeded" => 0
  "npending"           => 0
  "parallelism"        => 0
  "created"            => true
  "nuptodate"          => 0
  "succeeded"          => false

In [7]:
# Get the IP and port that was assigned for the webserver to listen on.
# The webserver may actually be running any of the physical nodes of the cluster.
# And it is running in an isolated container, with its own virtual IP address
# This IP address is accessible only to the user who started the webserver.
ip, portspec = @result getJobEndpoint(ctx, job)

2-element Array{Any,1}:
 "10.0.180.127"                                      
 Dict{String,Any}(Pair{String,Any}("port8000", 8000))

In [8]:
# We can connect to it from here (or anything else run by this user)
using Requests
url = "http://$(ip):$(first(values(portspec)))/"

"http://10.0.180.127:8000/"

In [9]:
String(Requests.bytes(get(url)))

"JuliaRun says hello from mywebserver-zrjdq"

In [10]:
# We stop the webserver now
@result deleteJob(ctx, job; force=true)

true